# 데이터 불러오기

## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import csv

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gb_module

## 기본 데이터

In [2]:
gb_df = pd.read_csv('result/gb_df.txt', sep = '\t')
gb_df.head(5)

,sent_id,sent_raw,sent_split,r_no,title,writer,w_new,ho_no,year,month,grid_1,period
0,1,創刊辭,창간 辭,1,創刊辭,-,uk01,1,1920,6,01q,p1
1,2,强者도 부르짖고 弱者도 부르짖으며 優者도 부르짖고 劣者도 부르짖도다,강자 약자 優者 劣者,1,創刊辭,-,uk01,1,1920,6,01q,p1
2,3,東西南北 四海八方이 다같이 소리中에 묻혀 있도다,동서 남북 사해 팔방 소리,1,創刊辭,-,uk01,1,1920,6,01q,p1
3,4,霹靂이냐 地震이냐 神籟이냐 魔哭이냐 우리는 아즉 이 소리의 正邪를 判斷할 수 없도다,소리 판단,1,創刊辭,-,uk01,1,1920,6,01q,p1
4,5,左右間 多數가 渴仰하고 多數가 要求하는 人民의 소리임은 明白하도다,좌우 間 다수 갈앙 다수 요구 인민 소리 명백,1,創刊辭,-,uk01,1,1920,6,01q,p1


# 시기별 rtm (논설 단위 tfidf)

In [29]:
periods = ['p1', 'p2', 'p3']
prd_df = {}  # 결과를 저장할 딕셔너리
prd_gisa = [] # 논설별 결과를 저장할 리스트

for i in periods:
    # 각 period에 대한 query 결과를 딕셔너리에 저장
    prd_df[f"{i}_gbdf"] = gb_df.query('period == @i')
#    print(prd_df[f"{i}_gbdf"].shape)
    
    # 각 시기 dtm 산출
    stopword = ['문제', '금일', '관계'] # 제외할 단어
    prd_dtm = gb_module.get_dtm(prd_df[f"{i}_gbdf"], 'sent_split', stopword, 50)
    
    # rtm_df 작성 (주요논설별 tfidf값) <- 기사 내 문서별 값을 평균
    merged_df = pd.merge(prd_dtm, prd_df[f"{i}_gbdf"], left_index=True, right_index=True, how='inner')
    merged_df1 = merged_df.iloc[:,0:54]
    merged_df2 = merged_df1.drop(['sent_id', 'sent_raw', 'sent_split'], axis=1)
    
    # r_no별로 평균값을 추출
    rtm_df = merged_df2.groupby('r_no').mean()
    prd_gisa.append(rtm_df)

In [30]:
a = prd_gisa[0].head(3) ; b = prd_gisa[1].head(3) ; c = prd_gisa[2].head(3)
print(a) ; print(b) ; print(c)

            사회       朝鮮        생활        사람        민족        自己        主義  \
r_no                                                                        
1     0.000000  0.00000  0.000000  0.298108  0.000000  0.000000  0.000000   
2     0.371296  0.00000  0.107415  0.365030  0.425303  0.862916  0.174445   
3     0.143822  0.23278  0.228840  0.181457  0.054914  0.027854  0.000000   

           사상        세계        문화  ...        현상        현재        일본  \
r_no                               ...                                 
1     0.00000  0.717898  0.000000  ...  0.000000  0.000000  0.000000   
2     0.04301  3.164610  0.091892  ...  0.000000  0.049984  0.000000   
3     0.00000  0.093639  0.032628  ...  0.035662  0.000000  0.037267   

            국가        인생        過去       농업        발달        지식        방법  
r_no                                                                       
1     0.000000  0.000000  0.000000  0.46099  0.000000  0.000000  0.000000  
2     0.735872  0.051210 

# 논설별 주요 단어 순위 등급화

In [28]:
# 각 시기별 데이터 처리 결과를 저장할 빈 DataFrame 초기화
final_df = pd.DataFrame()

for i, gisa in enumerate(prd_gisa):
    gisa1 = gisa.reset_index()

    # DataFrame을 긴 형태로 변환
    df_long = gisa1.melt(id_vars=['r_no'], var_name='단어', value_name='tf-idf')

    # TF-IDF 값이 0이 아닌 데이터만 필터링
    df_long_filtered = df_long[df_long['tf-idf'] > 0]

    # 다시 순위를 매김
    df_long_filtered = df_long_filtered.copy()
    df_long_filtered['rank'] = df_long_filtered.groupby('r_no')['tf-idf'].rank("dense", ascending=False).astype(int)

    # r_no와 순위에 따라 정렬
    df_long_filtered_sorted = df_long_filtered.sort_values(by=['r_no', 'rank'], ignore_index=True)

    # r_grade 열을 생성하고, rank 값에 따라 분류하여 값을 할당
    df_long_filtered_sorted['r_grade'] = pd.cut(df_long_filtered_sorted['rank'], bins=[0, 10, 20, 30, 40, 50], labels=['1rg', '2rg', '3rg', '4rg', '5rg'])
    
    # 시기 정보를 추가
    df_long_filtered_sorted['period'] = i+1

    # 시기별 결과를 최종 DataFrame에 추가
    final_df = pd.concat([final_df, df_long_filtered_sorted], ignore_index=True)
    
final_df

,r_no,단어,tf-idf,rank,r_grade,period
0,1,세계,0.717898,1,1rg,1
1,1,농업,0.460990,2,1rg,1
2,1,개조,0.396636,3,1rg,1
3,1,인류,0.385861,4,1rg,1
4,1,정신,0.367762,5,1rg,1
...,...,...,...,...,...,...
8374,334,정치,0.064705,25,3rg,3
8375,334,현상,0.037127,26,3rg,3
8376,334,무산,0.036492,27,3rg,3
8377,334,자유,0.035561,28,3rg,3


In [32]:
main_words = final_df.query('r_grade == "1rg"')
main_words

,r_no,단어,tf-idf,rank,r_grade,period
0,1,세계,0.717898,1,1rg,1
1,1,농업,0.460990,2,1rg,1
2,1,개조,0.396636,3,1rg,1
3,1,인류,0.385861,4,1rg,1
4,1,정신,0.367762,5,1rg,1
...,...,...,...,...,...,...
8355,334,운동,0.286021,6,1rg,3
8356,334,계급,0.246810,7,1rg,3
8357,334,主義,0.243127,8,1rg,3
8358,334,조직,0.239426,9,1rg,3


In [34]:
main_words.to_excel('result/1rank-grade_words.xlsx')

# The End of Note